In [30]:
#!pip install bert-for-tf2
#!pip install sentencepiece

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split




import tensorflow as tf

from numpy import loadtxt

import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

In [50]:



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [28]:
import numpy as np
import pandas as pd
import string

import transformers as ppb
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re as re

In [5]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [6]:
train_data['keyword'].fillna('no keyword', inplace = True) 
train_data['keyword'] = train_data['keyword'].str.replace('%20', ' ')
train_data['location'].fillna('no location', inplace = True)
train_data.drop(['id'],1,inplace = True)

In [7]:
#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

custom_sent_tokenizer = PunktSentenceTokenizer()

def process_content(sentence):
    tokenized = custom_sent_tokenizer.tokenize(sentence)
    words_tagged = []
    for i in tokenized:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        for word in tagged:
            words_tagged.append(word)
            
    return words_tagged

#Cleaning text

train_data['clean_text'] = train_data['text'].str.lower()

train_data['clean_text'] = train_data['clean_text'].apply(only_letters)    

#Tokenización

train_data['clean_text'] = train_data['clean_text'].apply(word_tokenize)

#Remove stopwords

train_data['clean_text'] = train_data['clean_text'].apply(filter_stopwords) 

#Lemmatization                                                                       

train_data['clean_text'] = train_data['clean_text'].apply(lemmatize_tweet)

#Transform to text
train_data['clean_text'] = train_data['clean_text'].apply(transform_to_text)

#Part of speech tagging

train_data['tagged_text']= train_data['text'].apply(process_content)
train_data['tagged_clean_text']= train_data['clean_text'].apply(process_content)

In [8]:
def count_Nouns(list):
    nouns = 0
    for word,tag in list:
        if tag[0] == 'N':
            nouns += 1
        else:
            continue
    return nouns

def count_Adjetives(list):
    adjetives = 0
    for word,tag in list:
        if tag[0] == 'J':
            adjetives += 1
        else:
            continue
    return adjetives

def count_Verbs(list):
    verbs = 0
    for word,tag in list:
        if tag[0] == 'V':
            verbs += 1
        else:
            continue
    return verbs 


# word_count
train_data['word_count'] = train_data['text'].apply(lambda x: len(str(x).split()))
test_data['word_count'] = test_data['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
train_data['unique_word_count'] = train_data['text'].apply(lambda x: len(set(str(x).split())))
test_data['unique_word_count'] = test_data['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
train_data['stop_word_count'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
test_data['stop_word_count'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

# url_count
train_data['url_count'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test_data['url_count'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
train_data['mean_word_length'] = train_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_data['mean_word_length'] = test_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# length
train_data['tweet_length'] = train_data['text'].apply(lambda x: len(str(x)))
test_data['tweet_length'] = test_data['text'].apply(lambda x: len(str(x)))

# punctuation_count
train_data['punctuation_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test_data['punctuation_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
train_data['hashtag_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
test_data['hashtag_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
train_data['mention_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
test_data['mention_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

#noun_count
train_data['noun_count'] = train_data['tagged_text'].apply(count_Nouns)
#test_data['mention_count'] = test_data[''].apply()

#verb_count
train_data['verb_count'] = train_data['tagged_text'].apply(count_Verbs)

#adjetives_count
train_data['adjetives_count'] = train_data['tagged_text'].apply(count_Adjetives)


In [9]:
train_data.head()

,keyword,location,text,target,clean_text,tagged_text,tagged_clean_text,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,tweet_length,punctuation_count,hashtag_count,mention_count,noun_count,verb_count,adjetives_count
0,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,"[(Our, PRP$), (Deeds, NNS), (are, VBP), (the, ...","[(deed, NN), (reason, NN), (earthquake, NN), (...",13,13,6,0,4.384615,69,1,1,0,6,1,0
1,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[(Forest, NNP), (fire, NN), (near, IN), (La, N...","[(forest, JJS), (fire, NN), (near, IN), (la, J...",7,7,0,0,4.571429,38,1,0,0,6,0,0
2,no keyword,no location,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...,"[(All, DT), (residents, NNS), (asked, VBD), (t...","[(resident, NN), (asked, VBD), (shelter, JJ), ...",22,20,11,0,5.090909,133,3,0,0,7,7,1
3,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...,"[(13,000, CD), (people, NNS), (receive, JJ), (...","[(people, NNS), (receive, VBP), (wildfire, NN)...",8,8,1,0,7.125000,65,2,1,0,4,1,1
4,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...,"[(Just, RB), (got, VBN), (sent, VBD), (this, D...","[(got, VBD), (sent, JJ), (photo, NN), (ruby, N...",16,15,7,0,4.500000,88,2,2,0,6,3,0


### Loading the Pre-trained BERT model


In [11]:
#DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

100%|██████████| 267967963/267967963 [00:23<00:00, 11302504.17B/s]


In [12]:
train_data_BERT = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [13]:

tokenized = train_data_BERT['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [14]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [15]:
np.array(padded).shape

(7613, 84)

In [18]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(7613, 84)

In [19]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [52]:
features = last_hidden_states[0][:,0,:].numpy()

In [53]:
len(features)


7613

In [54]:
X_train, X_test, y_train, y_test = train_test_split(features, train_data_BERT['target'].values, test_size=0.20, random_state=42)

In [43]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# define the keras model
model = keras.Sequential()
model.add(keras.layers.Dense(12, input_dim=768, activation='relu'))
#model.add(Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=12, batch_size=10)

Train on 6090 samples
Epoch 1/12
6090/6090 [==============================] - 2s 301us/sample - loss: 0.4697 - accuracy: 0.7780 - f1_m: 0.6828 - precision_m: 0.7808 - recall_m: 0.6615
Epoch 2/12
6090/6090 [==============================] - 1s 191us/sample - loss: 0.4232 - accuracy: 0.8087 - f1_m: 0.7247 - precision_m: 0.8342 - recall_m: 0.6846
Epoch 3/12
6090/6090 [==============================] - 1s 191us/sample - loss: 0.4169 - accuracy: 0.8108 - f1_m: 0.7304 - precision_m: 0.8248 - recall_m: 0.7007
Epoch 4/12
6090/6090 [==============================] - 1s 191us/sample - loss: 0.4105 - accuracy: 0.8174 - f1_m: 0.7359 - precision_m: 0.8354 - recall_m: 0.6984
Epoch 5/12
6090/6090 [==============================] - 1s 192us/sample - loss: 0.4034 - accuracy: 0.8215 - f1_m: 0.7466 - precision_m: 0.8449 - recall_m: 0.7141
Epoch 6/12
6090/6090 [==============================] - 1s 219us/sample - loss: 0.3983 - accuracy: 0.8222 - f1_m: 0.7438 - precision_m: 0.8483 - recall_m: 0.7058
Epoch 

In [44]:
# evaluate the keras model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))
print('f1: %.2f' % (f1_score*100))

1523/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Accuracy: 82.80
f1: 78.02
